**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
%cd /content/gdrive/My\ Drive/yolov3
!git clone https://github.com/AlexeyAB/darknet
%cd /content/gdrive/My\ Drive/yolov3/darknet

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
# %cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 10000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [10]:
!echo "pass" > data/obj.names
!echo "error1" >> data/obj.names
!echo "error2" >> data/obj.names
# !echo -e 'classes = 3\ntrain = data/train.txt\nvalid = data/val.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!echo "classes = 3" > data/obj.data
!echo "train = data/train.txt" >> data/obj.data
!echo "valid = data/val.txt" >> data/obj.data
!echo "names = data/obj.names" >> data/obj.data
!echo "backup = /mydrive/yolov3" >> data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

In [13]:
# Tạo file train.txt và val.txt
# %cd /content/gdrive/My\ Drive/yolov3/darknet

import glob2
import math  
import os
import numpy as np

files = []
for ext in ["*.png", "*.jpeg", "*.jpg"]:
  image_files = glob2.glob(os.path.join("data/obj/", ext))
  files += image_files

nb_val = math.floor(len(files)*0.2)
rand_idx = np.random.randint(0, len(files), nb_val)

# Tạo file train.txt
with open("data/train.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

# Tạo file vali.txt
with open("data/val.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx in rand_idx) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

In [ ]:
# %cd /content/gdrive/My\ Drive/yolov3/darknet
# !chmod +x ./darknet
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg yolov3_training_last.weights -dont_show 